In [1]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt

## obtain data

In [45]:
max_node = 18771 

data = [[] for i in range(max_node)] 
with open('./ca-AstroPh.txt', 'r') as f:
    f.readline() # skip the first row
    for line in f:
        node1, node2 = map(int, line.strip().split())
        data[node1 - 1].append(node2)
        data[node2 - 1].append(node1)

## define hash function

In [10]:
# def hashing funciton
def multiplicative_hash(element, a, scale=max_node):
    """
    Args:
        element: the element that will be hashed
        a: constant multiplier
        scale: the output scale
    """
    h = (element * a) % 1
    return int(h * scale)

## construct hash signature matrix

In [11]:
# %%timeit -r 1 -n 2

# the numbers of hash function
num = 500
signature_matrix = pd.DataFrame(data=np.zeros((num, max_node)),columns=np.arange(1, max_node+1))
rng = np.random.default_rng(num)

for i in range(num):
    # obtain multiplier randomly
    multiplier = rng.uniform(0,1)
    
    # store the minimum element of each node
    tmp_list = []
    
    # travese each node to find the min element 
    for node in range(max_node):
        tmp_list.append(np.min([ multiplicative_hash(data[node][_], multiplier) for _ in range(len(data[node])) ]))
       
    # assign the result of this hash function to a row of a signature matrix
    signature_matrix.loc[i] = tmp_list

In [12]:
signature_matrix

,1,2,3,4,5,6,7,8,9,10,...,18762,18763,18764,18765,18766,18767,18768,18769,18770,18771
0,43.0,384.0,24.0,10638.0,9.0,11.0,172.0,1643.0,75.0,4438.0,...,12541.0,6914.0,15046.0,1287.0,1287.0,9419.0,3792.0,11925.0,6298.0,14431.0
1,245.0,661.0,348.0,15028.0,78.0,159.0,245.0,308.0,157.0,245.0,...,9029.0,806.0,3547.0,11354.0,11354.0,14095.0,5872.0,8613.0,390.0,3131.0
2,113.0,83.0,11.0,12113.0,557.0,164.0,293.0,770.0,50.0,1313.0,...,10256.0,9056.0,15713.0,7856.0,2399.0,2399.0,13313.0,1199.0,18770.0,6656.0
3,271.0,762.0,517.0,7717.0,167.0,414.0,376.0,871.0,1.0,7717.0,...,13383.0,17765.0,10047.0,3377.0,3377.0,6712.0,41.0,11094.0,15477.0,7759.0
4,337.0,8.0,119.0,8785.0,121.0,365.0,52.0,400.0,280.0,1968.0,...,14027.0,2842.0,12827.0,1642.0,10427.0,1642.0,9227.0,441.0,8027.0,18012.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,286.0,268.0,342.0,6142.0,77.0,209.0,43.0,55.0,293.0,3042.0,...,17174.0,16830.0,10687.0,10343.0,4201.0,4201.0,9998.0,3856.0,3512.0,16141.0
496,969.0,1248.0,322.0,8326.0,165.0,1269.0,9.0,952.0,5.0,3517.0,...,5397.0,11606.0,3279.0,9489.0,1162.0,1162.0,15698.0,7371.0,13580.0,5253.0
497,258.0,2200.0,1641.0,9191.0,102.0,1109.0,94.0,344.0,208.0,1423.0,...,7774.0,16577.0,7386.0,6609.0,6609.0,6997.0,6221.0,15800.0,5832.0,15412.0
498,65.0,65.0,408.0,15487.0,919.0,128.0,186.0,1225.0,15.0,1922.0,...,8500.0,17421.0,1933.0,7571.0,7571.0,10854.0,1004.0,4288.0,13209.0,16492.0


##  banding technique(split the signature matrix into b bands, each with r rows)

In [16]:
b = 250

# create the buckets, which store the similar sets  
bucket = [{} for i in range(b)]

total_row = signature_matrix.shape[0]
r = total_row // b

 
# this hash function used to map from a band to a bucket
def int2connectedStr(ser):
    data_val = ""
    for i in range(ser.size):
        data_val += str(ser.iloc[i])
    return data_val

In [18]:
# %%timeit -r 1 -n 2

for i in range(b): # traverse band and establish buckets
    
    start = i * r
    end = (i + 1) * r - 1 if i != b - 1 else total_row - 1 
    
    # obtain the key of bucket of each set
    ser = signature_matrix.loc[start: end].apply(int2connectedStr)
    
    # traver the keys, and then put them into suitable bucket
    for index in ser.index:
        # if the key is not in bucket, just need to create a bucket
        if ser[index] not in bucket[i].keys():
            bucket[i][ser[index]] = []
        bucket[i][ser[index]].append(index)

## Searching

In [87]:
author = int(input("please input an author your want to search:"))

candidates = set()

for i in range(b):
    start = i * r
    end = (i + 1) * r - 1 if i != b - 1 else total_row - 1
    
    # acquire the band of the inputed author
    ser = signature_matrix.loc[start: end, author]
    
    # convert the content of the band to key
    val = int2connectedStr(ser)
    
    # add the other sets into candidates
    candidates |= set(bucket[i][val])
    
print()

print("candidates".center(32,'='))
candidates -= {author}
print(f"the number of candidates: {len(candidates)}")
print(candidates)

print()




please input an author your want to search:4382

===========candidates===========
the number of candidates: 115
{5120, 262, 3344, 3346, 6678, 5911, 3353, 3357, 4383, 14622, 5153, 4384, 4389, 3366, 2853, 1573, 14634, 3371, 3374, 14640, 14641, 1586, 4668, 3388, 318, 6210, 10052, 4677, 11588, 4679, 4425, 4682, 4684, 3405, 4685, 4431, 4686, 7509, 3413, 7511, 7512, 4693, 4694, 7515, 7516, 4695, 4700, 4703, 7518, 4705, 7522, 10595, 7523, 4707, 4710, 4711, 1386, 4460, 7537, 7538, 7539, 12660, 12662, 7542, 7545, 7550, 7553, 7554, 9858, 4225, 7555, 7558, 17030, 6280, 17031, 4234, 7562, 4995, 7566, 7567, 7570, 7572, 7573, 7575, 11927, 7577, 3479, 7591, 7592, 6318, 4017, 1716, 439, 2244, 2245, 1739, 973, 1999, 4306, 13780, 4309, 476, 4830, 2020, 1766, 489, 1259, 1005, 9454, 4334, 10479, 2547, 16380, 16381, 5374}



## validation

In [28]:
def jarccard(list1, list2):
    assert type(list1) == list and type(list2) == list and len(set(list1) | set(list2)) > 0
    return len(set(list1) & set(list2)) / len(set(list1) | set(list2))

In [74]:
# store the top 10 similar sets
estimate_top10 = {}

# filter sets from the candidates
for item in candidates:
    estimate_top10[item] = jarccard(data[item-1], data[author-1])
 
estimate_top10 = dict(sorted(estimate_top10.items(), key=lambda item:item[1], reverse=True)[:10])
estimate_top10

{15383: 0.4,
 14456: 0.25,
 17702: 0.2,
 10317: 0.15384615384615385,
 13548: 0.14285714285714285,
 8994: 0.13333333333333333,
 15384: 0.125,
 14454: 0.125,
 1794: 0.1111111111111111,
 15023: 0.1111111111111111}

In [80]:
ture_top10 = {}

# filter sets from the dataset
for item in range(max_node):
    if item != author - 1:
        ture_top10[item + 1] = jarccard(data[item], data[author-1])
        
ture_top10 = dict(sorted(ture_top10.items(), key=lambda item: item[1], reverse=True)[:10])
ture_top10

{15383: 0.4,
 14456: 0.25,
 17702: 0.2,
 10317: 0.15384615384615385,
 13548: 0.14285714285714285,
 8994: 0.13333333333333333,
 14454: 0.125,
 15384: 0.125,
 4305: 0.11538461538461539,
 1794: 0.1111111111111111}

In [88]:
def arrcuracy_rate(l1, l2):
    return len(set(l1) & set(l2)) / len(l1)
arrcuracy_rate(estimate_top10.keys(), ture_top10.keys())

0.9

## Amplifying a Locality-Sensitive Family

In [134]:
signature_matrix = signature_matrix.astype(np.int32)

In [135]:
and_result = np.bitwise_and(signature_matrix.loc[0::2, :].to_numpy(), signature_matrix.loc[1::2, :].to_numpy())
and_result.shape

(250, 18771)

In [136]:
or_result = np.bitwise_or(and_result[0::2, :], and_result[1::2, :])
or_result.shape

(125, 18771)